In [1]:
import pandas as pd 
import sqlite3

In [6]:
Players = pd.read_excel("Players.xlsx")
Teams = pd.read_excel("Teams.xlsx")
Games = pd.read_excel("Games.xlsx")
Player_Stats = pd.read_excel("Player_Stats.xlsx")

In [7]:
conn = sqlite3.connect("nba.db") 
Players.to_sql("Players", conn, if_exists = "append", index = False)
Teams.to_sql('Teams', conn, if_exists = "append", index = False)
Games.to_sql('Games', conn, if_exists = "append", index = False)
Player_Stats.to_sql('Player_Stats', conn, if_exists = "append", index = False)

756

In [8]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())
conn.close()
#Successfully created database of these dataframes

[('Players',), ('Teams',), ('Games',), ('Player_Stats',)]


## Question 1:
Find the average points per game for each player in the '2022' season who currently
plays for a team in the 'Western' conference. Include the player's full name and the
average points.

In [16]:
conn = sqlite3.connect('nba.db')
cursor = conn.cursor()

#SQL Query to find avg ppg for each player in 2022 season in western conference

# select full_name from Players and pts/ games_played from Player_Stats
# where conference = Western, join Players and Player_Stats on player_id, join Players and Teams on team_id
cmd = \
"""
SELECT Players.full_name, ROUND(CAST(Player_Stats.pts AS FLOAT) / CAST(Player_Stats.games_played AS FLOAT),1) AS points_per_game
FROM Players
JOIN Player_Stats ON Players.player_id = Player_Stats.player_id
JOIN Teams ON Players.team_id = Teams.team_id
WHERE Teams.conference = 'Western' AND Player_Stats.game_type = "regular";
"""
q1 = pd.read_sql_query(cmd, conn)
q1
q1.sort_values(by = 'points_per_game', ascending = False)

,full_name,points_per_game
62,Luka Doncic,32.4
54,Shai Gilgeous-Alexander,31.4
153,Stephen Curry,29.4
38,Kevin Durant,29.1
39,LeBron James,28.9
...,...,...
74,Alex Len,1.7
130,Keon Ellis,1.5
160,Wendell Moore Jr.,1.4
112,Braxton Key,1.3


## Question 2
Use a Common Table Expression (CTE) to calculate the total number of playoff games
played by each team in the '2022' season. Display the team's full name and the total
number of games played.


In [18]:
cmd = \
"""
WITH playoff_cte AS (
    SELECT Teams.team_id, Teams.team_full_name, COUNT(*) AS total_playoff_games
    FROM Teams
    JOIN Games ON Teams.team_id = Games.home_team_id OR Teams.team_id = Games.away_team_id
    WHERE Games.game_type = 'playoffs'
    GROUP BY Teams.team_id, Teams.team_full_name
)

SELECT team_full_name, total_playoff_games 
FROM playoff_cte
"""
q2 = pd.read_sql_query(cmd,conn)
q2

,team_full_name,total_playoff_games
0,Atlanta Hawks,6
1,Boston Celtics,20
2,Denver Nuggets,20
3,Cleveland Cavaliers,5
4,Golden State Warriors,13
5,Miami Heat,23
6,LA Clippers,5
7,Milwaukee Bucks,5
8,Los Angeles Lakers,16
9,Brooklyn Nets,4


## Question 3
Find the player who has the highest minutes per game in the '2022' playoffs, considering
players who have started at least 50% of the playoff games they played. Include the
player's full name and the average minutes per game.

In [30]:
# SELECT Players.full_name, WHERE player_stats.games_started / player_stats.games_played > 0.5
cmd = \
"""
SELECT Players.full_name, AVG(Player_Stats.minutes / Player_Stats.games_played) AS min_per_game
FROM Players
JOIN Player_Stats ON Players.player_id = Player_Stats.player_id
WHERE Player_Stats.game_type = 'playoffs' AND (CAST(Player_Stats.games_started AS FLOAT) / CAST(player_stats.games_played AS FLOAT) > 0.5)
ORDER BY min_per_game DESC
LIMIT 1;
"""
q3 = pd.read_sql_query(cmd,conn)
q3

,full_name,min_per_game
0,None,None


## Question 4
Retrieve the top 5 players with the highest average points per game in the '2022' regular
season, considering only players who have played for a team in the playoffs. Include the
player's full name, average points per game, and the team they are currently playing for.

In [45]:
# select players.full_name, afdfs as ppg, Teams.team_full_name
# 
cmd = \
"""
SELECT Players.full_name, ROUND(CAST(Player_Stats.pts AS FLOAT) / CAST(Player_Stats.games_played AS FLOAT),1) AS points_per_game, Teams.team_full_name
FROM Players
JOIN Teams ON Players.team_id = Teams.team_id 
JOIN Player_Stats ON Players.player_id = Player_Stats.player_id
JOIN Games ON Players.team_id = Games.home_team_id OR Players.team_id = Games.away_team_id
WHERE Player_Stats.game_type = 'regular' AND Games.game_type = 'playoffs'
GROUP BY Players.full_name
ORDER BY points_per_game DESC
LIMIT 5
;
"""
q4 = pd.read_sql_query(cmd, conn)
q4

,full_name,points_per_game,team_full_name
0,Joel Embiid,33.1,Philadelphia 76ers
1,Damian Lillard,32.2,Milwaukee Bucks
2,Giannis Antetokounmpo,31.1,Milwaukee Bucks
3,Jayson Tatum,30.1,Boston Celtics
4,Stephen Curry,29.4,Golden State Warriors
